#### _Speech Processing Labs 2020: TTS: Module 4_

In [ ]:
# run this first
import matplotlib.pyplot as plt
import numpy as np
import math
import IPython

# 2 Building a Decision Tree (using pencil and paper)

### Learning Outcomes
* Understand in detail how to choose the best partition of some data, to reduce entropy
* Be able to calculate entropy for distributions of real data
* Be able to grow a small decision tree without running any code

### Need to know
* what entropy is (from the previous notebook)
* Topic Videos: Prosody, Decision tree, Learning decision trees

**This exercise requires pencil and paper, so go get some! I mean it!**

We are going to do a few iterations of growing a Decision Tree by hand. This is not a handwritten tree though: we're running the normal algorithm, but we're going to perform it manually so we really understand every step. (Later, we'll run it in code.)

You should do this notebook in groups of 2-3 students.

Before you start, here's a short video to remind you how a decision tree is learned, to complement the topic videos.

In [ ]:
from IPython.display import HTML
IPython.display.IFrame(width="640",height="428",src="https://fast.wistia.net/embed/iframe/9l5iq63ezi")

## 2.1 Raw data

You are going to grow a small decision tree to perform the task of predicting where to place phrase breaks in a sentence, based only on the text. Fortunately, someone has already recorded some sentences, listened for where the speaker made phrase breaks, and labelled the data with those, so you don't need to. That's a relief! The raw data look like this, where "|" marks a phrase break:

```
You know a little too much, | Mr Hannay. |
It was obvious | that he was badly puzzled. |
In half an hour | I was reading. |
Then something awoke me. |
He stammered a little, | like a man picking his words. |
I pulled up a chair | and sat down on it. |
I tore open the Tide Tables | and found Bradgate. |
I took my head in my hands | and thought. |
He came back | in ten minutes | with a long face. |
He will be in London | at five. |
Then the dark fell, | and silence. |
But it was a chance, | the only possible chance. |
It was obvious | that he was badly puzzled. |
Then with some difficulty | I turned the car. |
```

## 2.2 Data preparation (feature extraction and feature engineering)

We rarely use the raw data directly in machine learning. Usually, we have some choices to make, using our linguistic knowledge and our skills as machine learning engineers.

### 2.2.1 Dealing with variable length sequences
As in so many problems involving a variable length sequence (here, the sentence), we reduce the problem to a fixed length sequence by using a sliding window. We need to do this because Decision Trees work with a fixed set of predictors, not a variable number.

The sliding window will be placed around each location where a break might occur: each word juncture.

### 2.2.2 Choosing the predictors
We also have to choose whether to use the raw predictor values (the orthographic words), or to process them in some way. Words are an open set with a very large numer of possible values, so the data will be very sparse. We will replace each word with a super-simple Part-Of-Speech (POS) tag:

    PUNC = punctuation
    CONT = content words
    FUNC = function words

That makes the data look like this, stored as list of sentences:

In [ ]:
corpus=[
    "FUNC CONT FUNC CONT CONT CONT PUNC | CONT CONT PUNC |",
    "FUNC CONT CONT | FUNC FUNC CONT CONT CONT PUNC |",
    "FUNC CONT FUNC CONT | FUNC CONT CONT PUNC |",
    "FUNC CONT CONT FUNC PUNC |",
    "FUNC CONT FUNC CONT PUNC | CONT FUNC CONT CONT FUNC CONT PUNC |",
    "FUNC CONT FUNC FUNC CONT | FUNC CONT CONT FUNC FUNC PUNC |",
    "FUNC CONT CONT FUNC CONT CONT | FUNC CONT CONT PUNC |",
    "FUNC CONT FUNC CONT FUNC FUNC CONT | FUNC CONT PUNC |",
    "FUNC CONT CONT | FUNC CONT CONT | FUNC FUNC CONT CONT PUNC |",
    "FUNC CONT FUNC FUNC CONT | FUNC CONT PUNC |",
    "FUNC FUNC CONT CONT PUNC | FUNC CONT PUNC |",
    "FUNC CONT CONT FUNC CONT PUNC | FUNC CONT CONT CONT PUNC |",
    "FUNC CONT CONT | FUNC FUNC CONT CONT CONT PUNC |",
    "FUNC FUNC FUNC CONT | FUNC CONT FUNC CONT PUNC |"]

### 2.2.3 What is the predictee?
The predictee always has to have a value. In the raw data, it is only defined at word junctures where there was a break. We will rewrite the predictee so it is either "NB" (no break) or "B" (break) at *every word juncture*.

## 2.3 Data prepared for machine learning

### 2.3.1 Nicely formatted data

From the raw data, we need to extract a set of data points for learning a decision tree. Each data point comprises the values of the predictors and the corresponding correct value of the predictee. This is the training data.

In [ ]:
data=[]
for line in corpus:
    # pad, ready for sliding window
    line="PAD "+line+" PAD"
    line=line.replace(" | ","-B-").replace(" "," NB ").replace("-B-"," B ")
    words=line.split()
    for i in range(0,len(words)):
        if words[i] == "B" or words[i] == "NB":
            data.append((words[i-1],words[i+1],words[i]))

print("Created",len(data),"data points. Each data point has the form (PREV, NEXT, PREDICTEE)")
for d in data:
    print("{}".format(d))

### 2.3.2 List all possible questions we can ask about the predictors

There are two predictors - the POS of the previous and next word surrounding a word juncture. We've called them PREV and NEXT. Each can take one of a fixed set of values. Write down all possible questions you can ask, in the box below. Don't worry about whether they are *actually useful* - that's not your job! (The data will tell us that.) I've given you one question as a starting point:

```
PREV=="FUNC"
```
...now edit this part to write down all other possible questions

## 2.4 Tree growing

### 2.4.1 Measure the entropy of the data before it is split

Do this by hand. You need to find the distribution of predictee values in the above data set, then apply the entropy equation. Do the calculation on paper and not in Python! The only part you can't easily do by hand is to compute logarithms, so here's a function for you to do that:

In [ ]:
p=0.5 # adjust this as required
print("{:.3}".format(math.log2(p)))

### 2.4.2 Pick a question and split the data, then measure the entropy of the two partitions

I'll start you off, and give you some code that will split the data for you. If you prefer (and you might learn more by doing this), you could print out the data on paper and cut it into 145 pieces, then do all of this without any code.

In [ ]:
yes=[]
no=[]
for d in data:
    # d[0] contains PREV and d[1] contains NEXT, with the predictee in d[2]
    if d[0] == "FUNC": # <- change this for each of your questions in turn
        yes.append(d)
    else:
        no.append(d)

yes_predictees = [d[2] for d in yes]
no_predictees = [d[2] for d in no]

print("Data for 'yes' is",yes,"\n")
print("Data for 'no'  is",no,"\n\n")
yes_predictees.sort()
no_predictees.sort()
print("Predictee distribution for 'yes' is",yes_predictees,"\n")
print("Predictee distribution for 'no'  is",no_predictees,"\n\n")
print("From a total of",len(data),"data points: 'yes' partition",len(yes),"/ 'no' partition",len(no))

Now we need to calculate the entropy of each of those two distributions, and compute the weighted sum. This will give the total entropy of the partitioned data.

The 'yes' one is easy - they are all 'NB' so the entropy is 0. You do the one for 'no'.

#### Repeat for all questions

Repeat step 2.4.2 for every one of of the questions you came up with in 2.3.2 (divide the labour amongst your group), recording the entropy for each one.

### 2.4.3 Place the best question into the tree

Once you have decided on the best question to split the whole data, write that as the root of the tree and draw a 'yes' and 'no' branch (really, do it on a piece of paper!). 

### 2.4.4 Recurse

Now recurse down one or both of those branches, treating each partition in turn as the training data. Your goal is to grow a *very* small tree. Keep going until you think you properly understand the algorithm.

## 2.5 Test

When you are finished, make up a new sentence and use your tree to predict where the phrase breaks should be placed.